# Category creation

In [11]:
import pandas as pd

In [12]:
# products_cl.csv
url = "https://drive.google.com/file/d/1s7Lai4NSlsYjGEPg1QSOUJobNYVsZBOJ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_csv(path)

product_category_df = products_cl.copy()

In [13]:
# creating category column
product_category_df = products_cl.copy()
product_category_df["category"] = ""

In [14]:
# Direct type mappings
type_category_map = {
    "11865403": "cases",
    "12175397": "servers",
    "1298": "open",
    "11935397": "drives",
    "11905404": "sensors",
    "1282": "Apple Imac",
    "12635403": "cases",
    "13835403": "macbook cases",
    "5,74E+15": "Apple iMac 27",
    "1364": "Mac Memory",
    "12585395": "adapter",
    "1325": "cables",
    "5384": "headphones",
    "1433": "expansion kits",
    "12215397": "SSD",
    "1,02E+12": "Apple MacBook Pro",
    "1,44E+11": "repair service",
    "57445397": "Memory Card",
    "12655397": "hard drive",
    "5395": "power extension",
}
# Keyword-based mappings
keyword_rules = [
    # keyword(s), column(s), label
    (["dock"], ["name", "desc"], "docking stations"),
    (["button"], ["name", "desc"], "button"),
    (["apple iphone"], ["name"], "apple smartphone"),
    (["apple ipod"], ["name"], "apple ipod"),
    (["apple ipad", "tablet"], ["name"], "apple tablet"),
    (["imac", "mac mini", "mac pro", "desktop computer"], ["name", "desc"], "mac desktop"),
    (["cable", "cables"], ["name"], "cables"),
    (["mouse"], ["name"], "mouse"),
    (["service"], ["name", "desc"], "installation"),
    (["repair"], ["name", "desc"], "repair"),
    (["replacement"], ["name"], "replacements"),
    (["hard drive"], ["name"], "hard drive"),
    (["strap"], ["name", "desc"], "strap"),
    (["protect", "glass"], ["name", "desc"], "screen protector"),
    (["keyboard"], ["desc"], "keyboards"),
    (["case"], ["name", "desc"], "cases"),
    (["switch"], ["name"], "switches"),
    (["car"], ["name"], "car accessories"),
    (["camera", "cameras"], ["name", "desc"], "camera"),
    (["kit"], ["desc"], "kits"),
    (["pen"], ["desc"], "pens"),
    (["battery", "batteries"], ["desc"], "battery"),
    (["support"], ["desc"], "support"),
    (["band", "sports", "fitness"], ["desc"], "sport"),
    (["satellite"], ["name"], "satellite"),
    (["adapter"], ["desc"], "adapter"),
    (["charger"], ["desc"], "charger"),
    (["license"], ["desc"], "license"),
    (["disk"], ["desc"], "disks"),
    (["expansion"], ["desc"], "expansion"),
    (["microphone"], ["desc"], "microphone"),
    (["router"], ["name", "desc"], "router"),
    (["software"], ["name"], "software"),
    (["pointer"], ["name", "desc"], "pointer"),
    (["bike"], ["name", "desc"], "bikes"),
    (["UPS"], ["name", "desc"], "UPS"),
]


In [15]:
# function for type

def apply_category_rules(df, type_map, keyword_rules):
    df["category"] = df["category"].fillna("")

    for t, cat in type_map.items():
        df.loc[df["type"] == t, "category"] += f", {cat}"

    for keywords, columns, cat in keyword_rules:
        mask = False
        for col in columns:
            for kw in keywords:
                mask |= df[col].str.contains(kw, case=False, na=False)
        df.loc[mask, "category"] += f", {cat}"
    return df

product_category_df = apply_category_rules(product_category_df, type_category_map, keyword_rules)


# --- 3️⃣ Special case: monitors (combined logic)
product_category_df.loc[
    (product_category_df["type"] == "1296")
    | (product_category_df["name"].str.contains("monitor", case=False, na=False))
    | (product_category_df["desc"].str.contains("monitor", case=False, na=False)),
    "category"
] += ", monitors"


In [16]:
# check
product_category_df[
    product_category_df['category'].isna() | (product_category_df['category'] == '')
]


,sku,name,desc,price,in_stock,type,category
28,SNN0007,Sonnet Tempo SATA PCIe Mac and PC E2P,PCIe Card with 2 Serial ATA ports Mac and PC.,59.99,0,1276,
38,JMO0026,Just Mobile Lazy Couch Support Mac and iPad,Mac and iPad small lift stand.,19.95,0,8696,
44,MOB0009,Mobee Magic Numpad for the Magic Trackpad,3 Numeric for Magic Trackpad stickers.,29.49,0,12355400,
58,MOS0065,Moshi TeraGlove cleaner screens,Microfibre screen cleaner for Mac iPhone iPad ...,20.00,0,12355400,
76,WAC0046,Intuos Wacom stylus ArtPen 5/4,special stylus Mac and PC for Intuos4 graphics...,109.90,0,1229,
...,...,...,...,...,...,...,...
9878,RIN0018,Ring Chime Pro Extender Wi-Fi and Timbre,Timbre increases your Wi-Fi signal and amplifi...,59.00,0,1334,
9903,BEL0301-A,Open - Belkin Base Qi Wireless charging up Boo...,Base induction charging with Qi technology for...,34.99,0,13615399,
9970,DLK0144,D-Link Wi-fi system COVR powerline mesh AC1200,Two network extenders Electric PLC Lonea Wi-Fi...,220.99,1,1334,
9985,KNO0032,"Knomo MacBook Pro Beauchamp Backpack 14 ""Black",Backpack thin nylon mesh internal compartment ...,179.00,1,1392,


In [17]:
# create "Others" category
product_category_df.loc[
    product_category_df['category'].isna() | (product_category_df['category'] == ''), "category"] = "Others"

In [18]:
# create key category
product_category_df["key category"] = ""

In [19]:
# create category map
category_map = {
    "tech products": ["mac desktop", "ipad", "tablet", "ipod", "watch", "smartphone", "laptop", "computer", "Apple Imac",
                      "Apple Macbook", "Apple iMac 27", "Apple MacBook Pro", "apple smartphone", "apple ipod", "apple tablet", "mac desktop" ],
    "accessories": ["cases", "microphone", "expansion", "sensors", "camera", "macbook cases", "adapter", "bikes",
                    "pointer", "monitors", "speakers", "cables", "headphones", "docking stations", "power extension",
                    "cables", "keyboards", "mouse", "charger", "cable", "adapter", "case", "strap", "switches", "car accessories",
                    "pens", "battery", "servers", "router", "support", "sport", "satellite", "expansion kits", "kits"],
    "services & spare parts": ["button", "screen protector", "repair service", "installation", "repair", "replacements"],
    "others": ["Others", "open"],
    "drives & memory": ["drives", "Mac Memory", "SSD", "Memory Card", "hard drive", "disks", "ssd"],
    "servers": ["servers", "router"],
    "systems & software": ["license", "software", "UPS"],
    }

In [20]:
for key_cat, subcats in category_map.items():
    mask = product_category_df['category'].str.contains('|'.join(subcats), case=False, na=False)

    # For each match
    for idx in product_category_df[mask].index:
        current_val = product_category_df.at[idx, 'key category']

        if pd.isna(current_val) or current_val == "":
            # Empty → assign directly
            product_category_df.at[idx, 'key category'] = key_cat
        elif key_cat not in current_val:
            # Non-empty → append if not already there
            product_category_df.at[idx, 'key category'] += f", {key_cat}"

In [21]:
# individual adaptation
product_category_df.loc[
    product_category_df["category"].str.contains("Mac Memory", case=False, na=False) &
    product_category_df["category"].str.contains("mac desktop", case=False, na=False),
    'key category'
] = "drives & memory"

product_category_df.loc[
    product_category_df["key category"].str.contains("tech products", case=False, na=False) &
    product_category_df["key category"].str.contains("accessories", case=False, na=False),
    'key category'
] = "accessories"

product_category_df.loc[
    product_category_df["key category"].str.contains("accessories", case=False, na=False) &
    product_category_df["key category"].str.contains("services & spare parts", case=False, na=False),
    'key category'
] = "services & spare parts"

In [22]:
categories_df = product_category_df.copy()